# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적잘한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습 시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다
5. D를 입력 받는다.

=>  
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.   
ex) A: 산, B: 바다, C: 나무, D: 물

**<출력 예시>**

관계 [수긍:추락 = 대사관:?]<br>
모델이 예측한 가장 적합한 단어: 잠입<br>
당신의 답변과 모델 예측의 유사도: 0.34<br>
아쉽네요. 더 생각해보세요.

In [ ]:
import re
from konlpy.tag import Okt
import fasttext

okt = Okt()

def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords

ko_stopwords = load_stopwords('ko_stopwords.txt')

def preprocess(text, return_tokens=False):
    text = re.sub("[^가-힣 ]", "", text)
    tokens = okt.morphs(text, stem=True)
    tokens = [word for word in tokens if word not in ko_stopwords]
    return tokens if return_tokens else ' '.join(tokens)


In [ ]:
load_txt = []
with open("naver_movie_ratings.txt", "r", encoding="utf-8") as f:
    next(f)  # 첫 줄(header) 건너뛰기
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) == 3:  # id, document, label
            _, document, _ = parts
            if document.strip():  # 비어있는 리뷰 제거
                load_txt.append(preprocess(document))

In [ ]:
with open("corpus.txt", "w", encoding="utf-8") as f:
    for line in load_txt:
        f.write(line + "\n")

In [ ]:
# skipgram 방식 (단어 임베딩 학습)
model = fasttext.train_unsupervised(
    'corpus.txt',
    model='skipgram',
    minCount=10,
    dim=100,
    minn=3,
    maxn=5,
)

# 유사 단어 찾기 테스트
print(model.get_nearest_neighbors("영화"))

[(0.9952020645141602, '영화인'), (0.9945622682571411, '영화관'), (0.9941626787185669, '명작'), (0.9940668940544128, '애니메이션'), (0.9939450621604919, '공포영화'), (0.9939148426055908, '이다'), (0.9938288331031799, '홍콩영화'), (0.9937629699707031, '엄청나다'), (0.993675172328949, '년전'), (0.9935327768325806, '평가')]


In [251]:
import random
import numpy as np

def find_random_similar_pair(model, threshold=0.8, max_trials=1000):
    words = model.get_words()
    for _ in range(max_trials):
        A = random.choice(words)
        similar_words = model.get_nearest_neighbors(A, k=20)  # [(score, word)]
        # threshold 이상인 단어 중 랜덤 선택
        candidates = [w for score, w in similar_words if score >= threshold and w != A]
        if candidates:
            B = random.choice(candidates)
            return A, B
    return None, None


In [252]:
def analogy_with_similarity(model, word_a, word_b, word_c, topn=5):
    """
    A : B = C : D 관계에서 D를 찾는 함수와 유사도 계산
    """
    try:
        # 입력 단어가 모델에 있는지 확인
        if not all(word in model.get_words() for word in [word_a, word_b, word_c]):
            raise KeyError("입력 단어 중 하나가 모델에 없습니다.")
        
        # A - B + C 벡터 계산
        vector = model.get_word_vector(word_b) - model.get_word_vector(word_a) + model.get_word_vector(word_c)
        
        # 벡터 정규화
        def normalize(v):
            norm = np.linalg.norm(v)
            return v / norm if norm > 0 else v
        
        vector = normalize(vector)
        
        # 모든 단어와의 유사도 계산
        words = model.get_words()
        similarities = [
            (word, np.dot(normalize(vector), normalize(model.get_word_vector(word))))
            for word in words
        ]
        
        # 유사도 기준으로 정렬
        similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
        
        # 입력 단어 제외
        candidates = [(word, similarity) for word, similarity in similarities if word not in {word_a, word_b, word_c}]
        
        # 상위 topn 단어 반환
        top_candidates = candidates[:topn]
        
        # A:B와 C:D 유사도 계산
        a_b_similarity = np.dot(
            normalize(model.get_word_vector(word_a)),
            normalize(model.get_word_vector(word_b))
        )
        c_d_similarity = np.dot(
            normalize(model.get_word_vector(word_c)),
            normalize(model.get_word_vector(top_candidates[0][0]))
        )
        
        return top_candidates, a_b_similarity, c_d_similarity
    except KeyError as e:
        print(f"단어를 찾을 수 없습니다: {e}")
        return [], None, None

In [253]:
A, B = find_random_similar_pair(model)
C = random.choice(model.get_words())
D_candidates, a_b_similarity, c_d_similarity = analogy_with_similarity(model, A, B, C, topn=5)

print(f"문제: {A} : {B} = {C} : ?")
if D_candidates:
    print("정답 후보:")
    for word, similarity in D_candidates:
        print(f"  {word} (유사도: {similarity:.4f})")
    print(f"\n유사도:")
    print(f"  {A}:{B} 유사도 = {a_b_similarity:.4f}")
    print(f"  {C}:{D_candidates[0][0]} 유사도 = {c_d_similarity:.4f}")
else:
    print("정답 후보를 찾을 수 없습니다.")

문제: 공감 : 외로움 = 뒤 : ?
정답 후보:
  계 (유사도: 0.9944)
  남녀 (유사도: 0.9944)
  작다 (유사도: 0.9943)
  외계인 (유사도: 0.9943)
  갈수록 (유사도: 0.9943)

유사도:
  공감:외로움 유사도 = 0.9987
  뒤:계 유사도 = 0.9945
